## 4 Brute attack

Брутфорс для рельсы транзакций (наивный скор на паблике ROC-AUC Diff 0,04825)

In [1]:
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
# from tqdm import tqdm
from model import predict # Функция, позволяет получить предсказание нейронки.
from check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt

bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются

train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target

In [2]:
seed = 26041999
BUDGET = 10 # разрешенное количество изменений транзакций для каждого пользователя - N транзакций
threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна

In [3]:
train_df = pd.read_csv(train_data_path)
train_df.head()

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,69,5541,48,-342.89792,2021-03-05 02:52:36
1,69,5533,48,-1251.88120,2021-03-05 09:43:28
2,69,5331,48,-87.30924,2021-03-05 11:17:23
3,69,5921,48,-1822.17700,2021-03-05 13:41:03
4,69,5311,48,-427.12363,2021-03-05 19:14:23


In [4]:
users_id = train_df.user_id.unique()

In [5]:
cutted_df = train_df.loc[(train_df.user_id == users_id[0]) | (train_df.user_id == users_id[1]) |
                         (train_df.user_id == users_id[2])]

In [6]:
cutted_df.loc[cutted_df.user_id == 196]

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
600,196,4131,48,-22.061630,2021-03-23 16:49:23
601,196,5814,48,-79.838520,2021-03-24 03:06:44
602,196,5814,48,-42.548330,2021-03-24 03:19:53
603,196,5814,48,-89.789540,2021-03-24 08:04:58
604,196,5411,48,-478.066100,2021-03-24 13:58:36
...,...,...,...,...,...
895,196,5411,48,-342.244700,2021-06-28 12:30:41
896,196,4131,48,-22.680431,2021-06-29 12:56:50
897,196,5411,48,-426.916720,2021-06-29 14:04:43
898,196,5814,48,-101.980470,2021-06-30 06:12:21


In [7]:
cutted_df.to_csv('temp_first.csv', index=False)

In [8]:
pd.read_csv(transactions_path).user_id.unique().shape

(4200,)

In [14]:
import pytorch_lightning as pl
import torch
import pickle
from model import process_for_nn, TransactionsDataset, get_dataloader, device, TransactionsRnn

pl.seed_everything(seed)
model = TransactionsRnn()
model.load_state_dict(torch.load(model_path))
model.to(device)
model.eval()
with open(bins_path, "rb") as f:
    bins = pickle.load(f)
# features = bins.pop("features")

def create_dl(df, bins_path):
    df_transactions = df.dropna().assign(
            hour=lambda x: x.transaction_dttm.dt.hour,
            day=lambda x: x.transaction_dttm.dt.dayofweek,
            month=lambda x: x.transaction_dttm.dt.month,
            number_day=lambda x: x.transaction_dttm.dt.day,
        )

    with open(bins_path, "rb") as f:
        bins = pickle.load(f)
    features = bins.pop("features")

    df = process_for_nn(df_transactions, features, bins)
    dataset = TransactionsDataset(df)
    dataloader = get_dataloader(dataset, device, is_validation=True)
    return dataloader


def predictor(dataloader, model):
    preds = []
    users = []
    for data, target in dataloader:
        y_pred = model(data)
        preds.append(y_pred.detach().cpu().numpy())
        users.append(target.detach().cpu().numpy())
    preds = np.concatenate(preds)
    users = np.concatenate(users)
    return pd.DataFrame({"user_id": users, "target": preds[:, 1]})

Global seed set to 26041999


In [15]:
def check_target_attack(user, target, attacked_target):
    if target.target_true.loc[target.user_id == user].values[0]:
        return target.target.loc[target.user_id == user].values[0] - attacked_target.target.loc[attacked_target.user_id == user].values[0]
    else:
        return attacked_target.target.loc[attacked_target.user_id == user].values[0] - target.target.loc[target.user_id == user].values[0]

In [16]:
cutted_df

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,69,5541,48,-342.897920,2021-03-05 02:52:36
1,69,5533,48,-1251.881200,2021-03-05 09:43:28
2,69,5331,48,-87.309240,2021-03-05 11:17:23
3,69,5921,48,-1822.177000,2021-03-05 13:41:03
4,69,5311,48,-427.123630,2021-03-05 19:14:23
...,...,...,...,...,...
895,196,5411,48,-342.244700,2021-06-28 12:30:41
896,196,4131,48,-22.680431,2021-06-29 12:56:50
897,196,5411,48,-426.916720,2021-06-29 14:04:43
898,196,5814,48,-101.980470,2021-06-30 06:12:21


In [32]:
bins_path = "nn_bins.pickle"
model_path = "nn_weights.ckpt"


cutted_df = pd.read_csv('datasets/sample_submission13.csv', index_col=0)
# cutted_df = pd.read_csv(transactions_path)
cutted_df['transaction_dttm'] = pd.to_datetime(cutted_df.transaction_dttm, format='%Y-%m-%d %H:%M:%S')

target = predict('datasets/sample_submission13.csv', bins_path, model_path)
# target = predict(transactions_path, bins_path, model_path)
target['target_true'] = target.target.apply(lambda x: 0 if x <= threshold else 1)

hero_user_train = 469439 # настоящий 0, предсказали 1
poor_user_train = 296241 # настоящий 1, предсказали 0


one_idx = target.index[target.target > threshold]  # индексы пользователей, которые предсказаны 1
zero_idx = target.index[target.target <= threshold] # индексы пользователей, которые предсказаны 0

users = target.user_id.values

one_users = users[one_idx] # предсказали 1
zero_users = users[zero_idx] # предсказали 0

for user in tqdm(users, ncols=cutted_df.user_id.unique().shape[0],
                 ascii=True, desc='Total'):
    # cutted_df_copy = cutted_df.copy()
    # cutted_df_copy = cutted_df.iloc[cutted_df.loc[cutted_df.user_id == user].index].copy() # берем только транзакции user для предикта
    cutted_df_copy = cutted_df.loc[cutted_df.user_id == user].copy()
    if user in one_users:
        copy_from = poor_user_train # похожим на Героя скопируем 10 последних транзакций Неудачника
    else:
        copy_from = hero_user_train # А похожим на Неудачника наоборот

    user_score = 0
    user_step_to = 0
    user_step_from = 0
    for step_to in tqdm(range(0, 300, 10), leave=False, desc='step_to'):
        idx_to = cutted_df_copy.index[cutted_df_copy.user_id == user][step_to:step_to+BUDGET] # айдишники транзакций юзера
        for step_from in tqdm(range(0, 300, 10), leave=False, desc='step_from'):
            idx_from = train_df.index[train_df.user_id == copy_from][step_from:step_from+BUDGET] # айдишники транзакций воннаби

            sign_to = np.sign(cutted_df_copy.loc[idx_to, "transaction_amt"].values)
            sign_from = np.sign(train_df.loc[idx_from, "transaction_amt"].values)
            sign_mask = (sign_to == sign_from)

            cutted_df_copy.loc[idx_to[sign_mask], "mcc_code"] = train_df.loc[idx_from[sign_mask], "mcc_code"].values
            cutted_df_copy.loc[idx_to[sign_mask], "transaction_amt"] = train_df.loc[idx_from[sign_mask], "transaction_amt"].values

            # temp_path = "temp.csv"
            # cutted_df_copy.to_csv(temp_path, index=False)
            # attacked_target = predict(temp_path, bins_path, model_path)

            dataloader = create_dl(cutted_df_copy, bins_path)

            attacked_target = predictor(dataloader, model)

            current_user_score = check_target_attack(user, target, attacked_target)

            if current_user_score >= user_score:
                user_score = current_user_score
                user_step_to = step_to
                user_step_from = step_from

    idx_to = cutted_df.index[cutted_df.user_id == user][user_step_to:user_step_to+BUDGET]
    idx_from = train_df.index[train_df.user_id == copy_from][user_step_from:user_step_from+BUDGET]
    sign_to = np.sign(cutted_df.loc[idx_to, "transaction_amt"].values)
    sign_from = np.sign(train_df.loc[idx_from, "transaction_amt"].values)
    sign_mask = (sign_to == sign_from)

    cutted_df.loc[idx_to[sign_mask], "mcc_code"] = train_df.loc[idx_from[sign_mask], "mcc_code"].values
    cutted_df.loc[idx_to[sign_mask], "transaction_amt"] = train_df.loc[idx_from[sign_mask], "transaction_amt"].values

output_path = "temp_last.csv"
cutted_df.to_csv(output_path, index=False)

Total:   0%|                                                                                                  …

step_to:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

...

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

step_from:   0%|          | 0/30 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [39]:
cutted_df.loc[cutted_df.user_id == user].user_id.unique()

array([63143], dtype=int64)

In [17]:
a = pd.read_csv('datasets/sample_submission0.csv', index_col=0)

In [18]:
a

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,66,5812,48,-1842.949000,2021-05-17 11:30:42
1,66,4112,48,-283.283720,2021-05-17 12:59:02
2,66,5814,48,-73.820390,2021-05-17 14:04:09
3,66,4111,48,10.971557,2021-05-17 15:03:10
4,66,7991,48,-214.796420,2021-05-17 15:33:43
...,...,...,...,...,...
89995,62431,5921,48,-384.078770,2021-05-06 15:21:07
89996,62431,5814,48,-121.755875,2021-05-07 08:58:01
89997,62431,5411,48,-723.543200,2021-05-08 13:21:54
89998,62431,5814,48,-296.336820,2021-05-15 08:54:59


In [21]:
b = pd.read_csv('datasets/sample_submission0_attacked.csv')

In [22]:
b

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,66,5812,48,-1842.949000,2021-05-17 11:30:42
1,66,4112,48,-283.283720,2021-05-17 12:59:02
2,66,5814,48,-73.820390,2021-05-17 14:04:09
3,66,4111,48,10.971557,2021-05-17 15:03:10
4,66,7991,48,-214.796420,2021-05-17 15:33:43
...,...,...,...,...,...
89995,62431,5921,48,-105.483200,2021-05-06 15:21:07
89996,62431,5411,48,-127.989006,2021-05-07 08:58:01
89997,62431,5411,48,-124.689440,2021-05-08 13:21:54
89998,62431,5411,48,-55.108986,2021-05-15 08:54:59


In [17]:
a2 = predict('datasets/sample_submission0.csv', bins_path, model_path)
b2 = predict('datasets/sample_submission0_attacked.csv', bins_path, model_path)

In [18]:
a2

,user_id,target
0,66,0.041682
1,30619,0.016743


In [19]:
b2

,user_id,target
0,66,0.019294
1,30619,0.017529


In [20]:
a2['true_target'] = a2.target.apply(lambda x: 0 if x <= threshold else 1)

In [21]:
a2

,user_id,target,true_target
0,66,0.041682,1
1,30619,0.016743,0


In [22]:
roc_auc_score(a2['true_target'], a2['target'])

1.0

In [23]:
roc_auc_score(a2['true_target'], b2['target'])

1.0

!!! разделить на 14 (число процессов) датасетов (по 300 пользователей в каждом)

In [24]:
transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются
t = pd.read_csv(transactions_path)

In [38]:
t.user_id.unique()[::6].shape

(700,)

In [54]:
t.user_id.unique()[::6] == t.user_id.values

C:\Users\zekat\AppData\Local\Temp\ipykernel_3872\2182086092.py:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  t.user_id.unique()[::6] == t.user_id.values


False

In [27]:
t[t['user_id'].isin(t.user_id.unique()[::150])]

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
0,66,5812,48,-1842.949000,2021-05-17 11:30:42
1,66,4112,48,-283.283720,2021-05-17 12:59:02
2,66,5814,48,-73.820390,2021-05-17 14:04:09
3,66,4111,48,10.971557,2021-05-17 15:03:10
4,66,7991,48,-214.796420,2021-05-17 15:33:43
...,...,...,...,...,...
1215295,837766,6012,48,-1861.603500,2021-07-05 17:54:12
1215296,837766,9222,48,-234.367280,2021-07-05 18:39:39
1215297,837766,6538,48,-9210.702000,2021-07-09 07:52:28
1215298,837766,6538,48,-4383.066400,2021-07-13 15:15:05


In [33]:
8400/300

28.0

In [32]:
t[t['user_id'].isin(t.user_id.unique()[::150])].user_id.unique()

array([    66,  30619,  63143,  96931, 129633, 161463, 191514, 223360,
       255524, 281362, 309584, 338977, 366426, 398573, 431079, 459496,
       489397, 521490, 551521, 582641, 611615, 644130, 673731, 708158,
       743700, 776923, 808488, 837766], dtype=int64)

In [ ]:
t = t[t['user_id'].isin(t.user_id.unique()[::12])]
t.shape

In [28]:
t.tail()

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
1259995,868873,5533,48,-458.10983,2021-06-27 09:25:53
1259996,868873,5411,48,-257.35254,2021-06-29 07:00:31
1259997,868873,5921,48,-446.21378,2021-06-30 02:26:42
1259998,868873,5411,48,-261.24158,2021-06-30 11:34:35
1259999,868873,5533,48,-169.16957,2021-07-02 07:22:38


In [26]:
t.iloc[90000*13:90000*(13+1)]

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
1170000,808488,6011,48,-12913.06300,2020-08-18 06:06:24
1170001,808488,5732,48,-266.99826,2020-08-18 06:50:43
1170002,808488,5499,48,-94.38989,2020-08-20 07:45:39
1170003,808488,5499,48,-99.58463,2020-08-21 09:26:51
1170004,808488,6011,48,-3740.74200,2020-08-24 00:48:09
...,...,...,...,...,...
1259995,868873,5533,48,-458.10983,2021-06-27 09:25:53
1259996,868873,5411,48,-257.35254,2021-06-29 07:00:31
1259997,868873,5921,48,-446.21378,2021-06-30 02:26:42
1259998,868873,5411,48,-261.24158,2021-06-30 11:34:35


In [31]:
import pandas as pd

transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются
t = pd.read_csv(transactions_path)
# t = t[t['user_id'].isin(t.user_id.unique()[::150])]
# split = t.shape[0] // 14
split = 90000
for i in range(14):
    t.iloc[split*i:split*(i+1)].to_csv(f'datasets/sample_submission{i}.csv', index=True)

    # 0 90k
    # 90k 180k
    # 180k 270k
    # ...
    # 1170k 1260k

In [1]:
import pandas as pd

In [30]:
ts = pd.read_csv('datasets/sample_submission13.csv', index_col=0)
print(ts.user_id.unique())
ts.loc[ts.user_id == ts.user_id.unique()[-1]]

[808488 808940 809017 809608 809828 809995 811473 811486 811718 811731
 811821 811829 811832 811907 812105 812166 812263 812512 812788 813041
 813135 813641 813690 813752 814083 814339 814425 814430 814663 814711
 814882 816086 816124 816337 816338 816417 816736 816739 816802 816881
 817150 817189 817275 817292 817565 817595 817669 818017 818263 818271
 818295 818333 818537 818660 819128 819135 819328 819348 819765 819875
 819963 820135 820443 820741 820744 820844 820959 821776 822074 822670
 822966 822968 823128 823169 823831 823941 824050 824490 824525 824623
 824777 824914 825376 825389 825841 825938 826217 826360 826572 826592
 826706 826816 826822 827372 827719 827856 828003 828181 828202 828227
 828265 828501 828768 828998 829037 829187 829240 829358 829571 829585
 829633 829814 829944 829974 830119 830448 830512 830695 830811 831314
 831545 831984 832160 832862 833132 833207 833225 833569 833712 833729
 834140 834348 834431 834522 834558 834834 834963 835487 835507 836148
 83653

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
1259700,868873,5533,48,-291.47003,2020-12-29 09:04:41
1259701,868873,5499,48,-434.34512,2020-12-30 02:52:13
1259702,868873,4112,48,-989.21814,2020-12-30 03:05:39
1259703,868873,4112,48,-980.56130,2020-12-30 03:30:27
1259704,868873,4112,48,-1187.31420,2020-12-30 03:34:57
...,...,...,...,...,...
1259995,868873,5533,48,-458.10983,2021-06-27 09:25:53
1259996,868873,5411,48,-257.35254,2021-06-29 07:00:31
1259997,868873,5921,48,-446.21378,2021-06-30 02:26:42
1259998,868873,5411,48,-261.24158,2021-06-30 11:34:35


In [20]:
'datasets/sample_submission0.csv'[:-4] + '_attacked.csv'

'datasets/sample_submission0_attacked.csv'

In [17]:
df_attacked1 = pd.read_csv('datasets/sample_submission0_attacked.csv')
df_attacked2 = pd.read_csv('datasets/sample_submission1_attacked.csv')
df_attacked3 = pd.read_csv('datasets/sample_submission2_attacked.csv')
df_attacked4 = pd.read_csv('datasets/sample_submission3_attacked.csv')

In [23]:
df_attacked = pd.concat([df_attacked1, df_attacked2, df_attacked3, df_attacked4], ignore_index=True)
attacked_path = "brute_atack.csv"
df_attacked.to_csv(attacked_path, index=False)

In [20]:
transactions_path = "../sample_submission.csv"    # путь до файла с транзакциями, которые атакуются
t = pd.read_csv(transactions_path)

In [27]:
from model import predict # Функция, позволяет получить предсказание нейронки.
from check_budget import check_budget # функция проверки бюджета. Проверяйте допустимость решения до сабмита
from sklearn.metrics import roc_auc_score
bins_path = "nn_bins.pickle" # путь до файла с бинами после тренировки модели (nn_bins.pickle)
model_path = "nn_weights.ckpt" # путь до файла с весами нейронной сети (nn_weights.ckpt)
quantiles_path = "quantiles.json" # путь до файла с квантилями для таргета (quantiles.pickle)

In [26]:
train_data_path = "../transactions_finetune.csv" # X - train data
train_target_path = "../target_finetune.csv" # y - true target
threshold = pd.read_csv(train_target_path).target.mean() # примерный threshold трейна

In [30]:
pred = predict('../sample_submission.csv', bins_path, model_path, random_seed=26041999)
pred['target_true'] = pred.target.apply(lambda x: 0 if x <= threshold else 1)
pred

,user_id,target,target_true
0,66,0.030193,0
1,213,0.015984,0
2,345,0.103124,1
3,569,0.040291,1
4,573,0.029942,0
...,...,...,...
4195,868183,0.023982,0
4196,868407,0.044375,1
4197,868573,0.039456,1
4198,868694,0.022834,0


In [31]:
pred_attacked = predict(attacked_path, bins_path, model_path, random_seed=26041999)
pred_attacked

,user_id,target
0,66,0.034895
1,213,0.067802
2,345,0.088202
3,569,0.081078
4,573,0.053403
...,...,...
4195,868183,0.029341
4196,868407,0.032024
4197,868573,0.015489
4198,868694,0.015818


In [32]:
roc_auc_score(pred['target_true'], pred['target'])

1.0

In [33]:
roc_auc_score(pred['target_true'], pred_attacked['target'])

0.8464579775936287

In [34]:
check_budget(transactions_path, attacked_path, quantiles_path) # Не забываем проверять бюджет перед самбитом!

100%|█████████████████████████████████████████████████████████████████████| 1260000/1260000 [00:34<00:00, 36878.17it/s]


True

In [35]:
attacked_path

'brute_atack.csv'

In [36]:
best_path = 'naive_submission2.csv'
df_best = pd.read_csv(best_path)

In [56]:
pred_best = predict(best_path, bins_path, model_path, random_seed=26041999)

In [57]:
# best score
roc_auc_score(pred['target_true'], pred_best['target'])

0.7376394456315845

In [ ]:
# айдишники в сабмите last 10
# 829187 true_pred 1
# 488752 true_pred 0
# hero_user = 531067
# poor_user = 830119

In [71]:
pred.loc[pred.user_id == 488752]

,user_id,target,target_true
2394,488752,0.001358,0


In [74]:
pred.loc[pred.target_true == 0].sort_values('target')

,user_id,target,target_true
4014,830119,0.000427,0
1098,233068,0.000558,0
346,73615,0.000658,0
1317,276759,0.000672,0
1526,313769,0.000865,0
...,...,...,...
737,158492,0.036854,0
2242,458250,0.036861,0
4100,848849,0.036957,0
3082,630649,0.036973,0


In [62]:
pred.loc[pred.user_id == 829187]

,user_id,target,target_true
4005,829187,0.231564,1


In [63]:
pred.loc[pred.user_id == 488752]

,user_id,target,target_true
2394,488752,0.001358,0


In [65]:
pred_best.loc[pred_best.user_id == 829187]

,user_id,target
2394,488752,0.006837


In [59]:
t.loc[t.user_id == 829187]

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
1201500,829187,5331,48,-206.96848,2020-11-25 12:27:10
1201501,829187,5331,48,-153.09885,2020-11-26 02:07:45
1201502,829187,5331,48,-342.14197,2020-11-26 03:57:40
1201503,829187,5331,48,-152.63141,2020-11-27 01:54:18
1201504,829187,5331,48,-181.54138,2020-11-27 06:22:04
...,...,...,...,...,...
1201795,829187,6012,48,-934.84080,2021-07-01 12:31:01
1201796,829187,5499,48,-194.03397,2021-07-06 09:32:50
1201797,829187,5331,48,-177.24716,2021-07-08 02:35:52
1201798,829187,5411,48,-174.05374,2021-07-12 09:07:15


In [60]:
df_best.loc[df_best.user_id == 829187]

,user_id,mcc_code,currency_rk,transaction_amt,transaction_dttm
1201500,829187,5331,48,-206.96848,2020-11-25 12:27:10
1201501,829187,5331,48,-153.09885,2020-11-26 02:07:45
1201502,829187,5331,48,-342.14197,2020-11-26 03:57:40
1201503,829187,5331,48,-152.63141,2020-11-27 01:54:18
1201504,829187,5331,48,-181.54138,2020-11-27 06:22:04
...,...,...,...,...,...
1201795,829187,6012,48,-934.84080,2021-07-01 12:31:01
1201796,829187,5499,48,-194.03397,2021-07-06 09:32:50
1201797,829187,5331,48,-177.24716,2021-07-08 02:35:52
1201798,829187,5411,48,-174.05374,2021-07-12 09:07:15
